In [1]:
import pandas as pd
train = pd.read_csv('Data/train.csv',encoding = "ISO-8859-1")
test = pd.read_csv('Data/test.csv',encoding = "ISO-8859-1")

In [2]:
train.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99989 entries, 0 to 99988
Data columns (total 3 columns):
ItemID           99989 non-null int64
Sentiment        99989 non-null int64
SentimentText    99989 non-null object
dtypes: int64(2), object(1)
memory usage: 2.3+ MB


In [4]:
train['Sentiment'].value_counts()

1    56457
0    43532
Name: Sentiment, dtype: int64

In [42]:
import string
import re
punctuation = string.punctuation
def clean(sentence):
    sentence = sentence.lower()
    sentence = sentence.strip('#')
    sentence = re.sub(r'http\S+', '', sentence)
    sentence = ''.join(ch for ch in sentence if ch not in set(punctuation))
    sentence = ''.join([i for i in sentence if not i.isdigit()])
    sentence = ' '.join(sentence.split())
    return sentence

In [43]:
train['SentimentText'] = train['SentimentText'].apply(clean)

In [44]:
train['SentimentText'].head()

0                          is so sad for my apl friend
1                        i missed the new moon trailer
2                                    omg its already o
3    omgaga im sooo im gunna cry ive been at this d...
4                   i think mi bf is cheating on me tt
Name: SentimentText, dtype: object

In [45]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [46]:
WNL = WordNetLemmatizer()
def lemma(sentence):
    s = list()
    for x in sentence.split():
        s.append(WNL.lemmatize(x))
    return ' '.join(s)

In [58]:
train['SentimentText'] = train['SentimentText'].apply(lemma)

In [67]:
train.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my apl friend
1,2,0,i missed the new moon trailer
2,3,1,omg it already o
3,4,0,omgaga im sooo im gunna cry ive been at this d...
4,5,0,i think mi bf is cheating on me tt


In [61]:
import tensorflow as tf
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, Conv2D, Dropout, GRU, MaxPooling2D, Embedding

In [73]:
model = Sequential()
model.add(Embedding(20000, 128, input_length=100))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [74]:
X = train['SentimentText']
y = train['Sentiment']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.1)
model.fit(x_train, y_train,
          batch_size=32,
          epochs=4,
          validation_data=[x_test, y_test])

ValueError: Error when checking input: expected embedding_3_input to have shape (100,) but got array with shape (1,)